# Configurações Pyspark

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

import pandas as pd

def write_to_postgresl(df, tb_name=None, write_mode='None'):
    df.count()
    if tb_name is None:
        raise Exception('Informe o nome da tabela')
    if write_mode is None:
        raise Exception('Informe o mode de escrita: append ou overwrite')
        
    from datetime import datetime
    start_time = datetime.now()
    
    try:
        df.write.jdbc(url, table=tb_name, mode=write_mode, properties=properties)
    except Exception as e:
        print(f'Erro: {e}')
    end_time = datetime.now()
    
    total_time = str(end_time - start_time)
    
    return f'Total time: {total_time} and - Total rows: {df.count()} - Total columns: {len(df.columns)}'



    # Criando a sessão do Spark
spark  = SparkSession.builder \
    .appName("Data Analysis") \
    .config('spark.jars', '/data/IDAF/DATABASECONNECTOR_JAR_FOLDER/postgresql-42.2.18.jar')\
    .config("spark.executor.memory", "8g")\
    .config("spark.driver.memory", "8g")\
    .config("spark.executor.cores", "4")\
    .config("spark.executor.instances","8")\
    .config("spark.sql.shuffle.partitions","96")\
    .config("spark.default.parallelism","96")\
    .getOrCreate()


url = "jdbc:postgresql://localhost:5432/postgres"

properties = {
    "user" : "postgres",
    "password" : "cidacs",
    "driver" : "org.postgresql.Driver"
}

# Configurações Pandas

In [9]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100)
spark.conf.set("spark.sql.repl.eagerEval.maxNumRows", 300)

spark.conf.set("spark.sql.debug.maxToStringFields", 100)

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_rows", None)

### Lendo dados enriquecidos

In [10]:
df_input = (spark
            .read
            .parquet('/data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/basefinal_tb_limp_enriched')
           )

### Observation Period

In [11]:
window_observation_period = Window.partitionBy().orderBy('person_id', 'dtnasc_sinasc')

In [12]:
df_observation_period = (df_input
                         .withColumn('person_id', F.col('person_id'))
                         .withColumn('observation_period_start_date', F.when(F.col('mesprenat_sinasc')<=9, F.add_months(F.col('dtnasc_sinasc'), -(9 - F.col('mesprenat_sinasc').cast('int'))))
                                     .when((F.col('mesprenat_sinasc')==99)|(F.col('mesprenat_sinasc').isNull()), F.lit('2099-12-31'))
                                     .when(F.col('mesprenat_sinasc')>9, F.trunc(F.col('dtnasc_sinasc'), 'month')))
                         .withColumn('observation_period_end_date', F.col('dtnasc_sinasc'))
                         .withColumn('period_type_concept_id', F.lit(32879))
                         .withColumn('observation_period_id', F.row_number().over(window_observation_period))
                        ).select(F.col('observation_period_id').cast('integer'),
                                 F.col('person_id').cast('integer'),
                                 F.col('observation_period_start_date').cast('date'),
                                 F.col('observation_period_end_date').cast('date'),
                                 F.col('period_type_concept_id').cast('integer')
                                )

df_observation_period.count()

16590554

In [13]:
# df_observation_period.select(F.max('observation_period_id')).toPandas()

## Salvando no Postgresql

In [14]:
write_to_postgresl(df_observation_period, tb_name='tuberculose.observation_period', write_mode='append')

25/07/18 09:29:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/18 09:29:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/18 09:29:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/18 09:29:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/18 09:29:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


'Total time: 0:09:44.185629 and - Total rows: 16590554 - Total columns: 5'